# PyCity Schools Analysis

While looking through the results, a few conclusions are evident.

1- There is actually a negative correlation between school spending and test scores within this school district.

2- Medium schools had the greatest percentage of their students pass both math and reading.

3- Charter Schools outperformed district schools in their average scores and the percentage of their students that passed. It is unclear whether this is due to lesson structure or student to teacher ratio.

In [1]:
# Dependencies and Setup
import pandas as pd
from pathlib import Path

# File to Load (Remember to Change These)
school_data_to_load = Path("Resources/schools_complete.csv")
student_data_to_load = Path("Resources/students_complete.csv")

# Read School and Student Data File and store into Pandas DataFrames
school_data_df = pd.read_csv(school_data_to_load)
student_data_df = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete_df = pd.merge(student_data_df, school_data_df, how="left", on=["school_name", "school_name"])
school_data_complete_df.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


## District Summary

In [2]:
#Show column types
school_data_complete_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39170 entries, 0 to 39169
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Student ID     39170 non-null  int64 
 1   student_name   39170 non-null  object
 2   gender         39170 non-null  object
 3   grade          39170 non-null  object
 4   school_name    39170 non-null  object
 5   reading_score  39170 non-null  int64 
 6   math_score     39170 non-null  int64 
 7   School ID      39170 non-null  int64 
 8   type           39170 non-null  object
 9   size           39170 non-null  int64 
 10  budget         39170 non-null  int64 
dtypes: int64(6), object(5)
memory usage: 3.3+ MB


In [3]:
# Calculate the total number of unique schools
school_count = school_data_complete_df["school_name"].nunique()
school_count

15

In [4]:
# Calculate the total number of students
student_count = school_data_complete_df["student_name"].count()
student_count

39170

In [5]:
# Calculate the total budget
budget_df = school_data_complete_df["budget"].unique()
total_budget = budget_df.sum()

total_budget

24649428

In [6]:
# Calculate the average (mean) math score
average_math_score = school_data_complete_df["math_score"].mean()
average_math_score

78.98537145774827

In [7]:
# Calculate the average (mean) reading score
average_reading_score = school_data_complete_df["reading_score"].mean()
average_reading_score

81.87784018381414

In [8]:
# Use the following to calculate the percentage of students who passed math (math scores greather than or equal to 70)
passing_math_count = school_data_complete_df[(school_data_complete_df["math_score"] >= 70)].count()["student_name"]
passing_math_percentage = passing_math_count / float(student_count) * 100
passing_math_percentage

74.9808526933878

In [9]:
# Calculate the percentage of students who passed reading (hint: look at how the math percentage was calculated)  
passing_reading_count = school_data_complete_df[(school_data_complete_df["reading_score"] >= 70)].count()["student_name"]
passing_reading_percentage = passing_reading_count / float(student_count) * 100
passing_reading_percentage

85.80546336482001

In [10]:
# Use the following to calculate the percentage of students that passed math and reading
passing_math_reading_count = school_data_complete_df[
    (school_data_complete_df["math_score"] >= 70) & (school_data_complete_df["reading_score"] >= 70)
].count()["student_name"]
overall_passing_rate = passing_math_reading_count /  float(student_count) * 100
overall_passing_rate

65.17232575950983

In [11]:
# Create a high-level snapshot of the district's key metrics in a DataFrame
district_summary = pd.DataFrame({'Total Schools':school_count, 'Total Students':student_count, 'Total Budget':total_budget,
                             'Average Math Score':average_math_score, 'Average Reading Score':average_reading_score, 
                             '% Passing Math':passing_math_percentage, '% Passing Reading':passing_reading_percentage,
                             '% Overall Passing':overall_passing_rate}, index=[0])

# Formatting
district_summary["Total Students"] = district_summary["Total Students"].map("{:,}".format)
district_summary["Total Budget"] = district_summary["Total Budget"].map("${:,}".format)

# Display the DataFrame
district_summary

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428",78.985371,81.87784,74.980853,85.805463,65.172326


## School Summary

In [12]:
# Use the code provided to select all of the school types
# with column names. Groubpy isn't working
school_types = school_data_complete_df.groupby('school_name')['type'].min()
#school_types.groupby('school_name','type')

# without column names
#school_types = school_data_complete_df.set_index('school_name')["type"]
#school_types = school_types.groupby('school_name').unique()
school_types = pd.DataFrame(school_types).reset_index()
school_types

,school_name,type
0,Bailey High School,District
1,Cabrera High School,Charter
2,Figueroa High School,District
3,Ford High School,District
4,Griffin High School,Charter
5,Hernandez High School,District
6,Holden High School,Charter
7,Huang High School,District
8,Johnson High School,District
9,Pena High School,Charter


In [13]:
# Calculate the total student count per school
per_school_counts = school_data_complete_df.groupby('school_name', as_index=False)["student_name"].count()
per_school_counts = per_school_counts.rename(columns={"student_name": 'student_count'})

#per_school_counts = pd.DataFrame(per_school_counts)
# ^Reinstate line above to convert from Series to DF

per_school_counts

,school_name,student_count
0,Bailey High School,4976
1,Cabrera High School,1858
2,Figueroa High School,2949
3,Ford High School,2739
4,Griffin High School,1468
5,Hernandez High School,4635
6,Holden High School,427
7,Huang High School,2917
8,Johnson High School,4761
9,Pena High School,962


In [14]:
# Calculate the total school budget per school
per_school_budget = school_data_complete_df.groupby('school_name', as_index=False)[('budget')].min()
per_school_budget = per_school_budget.rename(columns={'budget': 'school_budget'})

#Formatting. May need to omit this here for calculating budget per capita and format after calculations.
#per_school_budget["budget"] = per_school_budget["budget"].astype({'budget': 'int64'}).map("${:,}".format)
# ^Reinstate line above to format DF

per_school_budget

,school_name,school_budget
0,Bailey High School,3124928
1,Cabrera High School,1081356
2,Figueroa High School,1884411
3,Ford High School,1763916
4,Griffin High School,917500
5,Hernandez High School,3022020
6,Holden High School,248087
7,Huang High School,1910635
8,Johnson High School,3094650
9,Pena High School,585858


In [15]:
# Calculate the total per capita spending per school
per_school_capita = pd.DataFrame({'School Name':school_types['school_name'],
                                    'Per Student Capita': per_school_budget['school_budget']
                                          / per_school_counts['student_count']})
per_school_capita

,School Name,Per Student Capita
0,Bailey High School,628.0
1,Cabrera High School,582.0
2,Figueroa High School,639.0
3,Ford High School,644.0
4,Griffin High School,625.0
5,Hernandez High School,652.0
6,Holden High School,581.0
7,Huang High School,655.0
8,Johnson High School,650.0
9,Pena High School,609.0


In [16]:
# Calculate the average math test scores per school
per_school_math = school_data_complete_df.groupby('school_name', as_index=False)["math_score"].mean()
per_school_math

,school_name,math_score
0,Bailey High School,77.048432
1,Cabrera High School,83.061895
2,Figueroa High School,76.711767
3,Ford High School,77.102592
4,Griffin High School,83.351499
5,Hernandez High School,77.289752
6,Holden High School,83.803279
7,Huang High School,76.629414
8,Johnson High School,77.072464
9,Pena High School,83.839917


In [17]:
# Calculate the average reading test scores per school
per_school_reading = school_data_complete_df.groupby('school_name', as_index=False)["reading_score"].mean()
per_school_reading

,school_name,reading_score
0,Bailey High School,81.033963
1,Cabrera High School,83.975780
2,Figueroa High School,81.158020
3,Ford High School,80.746258
4,Griffin High School,83.816757
5,Hernandez High School,80.934412
6,Holden High School,83.814988
7,Huang High School,81.182722
8,Johnson High School,80.966394
9,Pena High School,84.044699


In [18]:
# Calculate the number of students per school with math scores of 70 or higher
students_passing_math = school_data_complete_df[school_data_complete_df["math_score"] >= 70]
students_passing_math = students_passing_math.groupby('school_name', as_index=False)["student_name"].count()

# Rename student_name column to numerical
students_passing_math = students_passing_math.rename(columns={'student_name': '# Passing Math'})
students_passing_math

# Source for code below: https://github.com/racquesta/PyCitySchools/blob/master/PyCitySchools.ipynb
# % passing scores
# pass_math = merged_df[merged_df['math_score'] >= 70].groupby('school')['Student ID'].count()/stu_per_sch

,school_name,# Passing Math
0,Bailey High School,3318
1,Cabrera High School,1749
2,Figueroa High School,1946
3,Ford High School,1871
4,Griffin High School,1371
5,Hernandez High School,3094
6,Holden High School,395
7,Huang High School,1916
8,Johnson High School,3145
9,Pena High School,910


In [19]:
# Calculate the number of students per school with reading scores of 70 or higher
students_passing_reading = school_data_complete_df[school_data_complete_df["reading_score"] >= 70]
students_passing_reading = students_passing_reading.groupby('school_name', as_index=False)["student_name"].count()

# Rename student_name column to numerical
students_passing_reading = students_passing_reading.rename(columns={'student_name': '# Passing Reading'})
students_passing_reading

,school_name,# Passing Reading
0,Bailey High School,4077
1,Cabrera High School,1803
2,Figueroa High School,2381
3,Ford High School,2172
4,Griffin High School,1426
5,Hernandez High School,3748
6,Holden High School,411
7,Huang High School,2372
8,Johnson High School,3867
9,Pena High School,923


In [20]:
# Use the provided code to calculate the number of students per school that passed both math and reading with scores of 70 or higher
students_passing_math_and_reading = school_data_complete_df[
    (school_data_complete_df["reading_score"] >= 70) & (school_data_complete_df["math_score"] >= 70)]

# Groupby school
school_students_passing_math_and_reading = students_passing_math_and_reading.groupby("school_name", as_index=False).size()

# Rename size column to numerical
school_students_passing_math_and_reading = school_students_passing_math_and_reading.rename(
                                            columns={'size': '# Passing Both'})
# Display count of students
school_students_passing_math_and_reading

,school_name,# Passing Both
0,Bailey High School,2719
1,Cabrera High School,1697
2,Figueroa High School,1569
3,Ford High School,1487
4,Griffin High School,1330
5,Hernandez High School,2481
6,Holden High School,381
7,Huang High School,1561
8,Johnson High School,2549
9,Pena High School,871


In [21]:
# Use the provided code to calculate the passing rates
per_school_passing_math = pd.DataFrame({'School Name':school_types['school_name'],
                                      '% Passing Math': students_passing_math['# Passing Math'] /
                                        per_school_counts['student_count'] * 100})
per_school_passing_reading = pd.DataFrame({'School Name':school_types['school_name'],
                                      '% Passing Reading': students_passing_reading['# Passing Reading'] /
                                           per_school_counts['student_count'] * 100})
overall_passing_rate = pd.DataFrame({'School Name':school_types['school_name'],
                                      '% Passing Both': school_students_passing_math_and_reading['# Passing Both'] /
                                           per_school_counts['student_count'] * 100})
overall_passing_rate

,School Name,% Passing Both
0,Bailey High School,54.642283
1,Cabrera High School,91.334769
2,Figueroa High School,53.204476
3,Ford High School,54.289887
4,Griffin High School,90.599455
5,Hernandez High School,53.527508
6,Holden High School,89.227166
7,Huang High School,53.513884
8,Johnson High School,53.539172
9,Pena High School,90.540541


In [22]:
# Create a high-level snapshot of the district's key metrics in a DataFrame

# Create a DataFrame called `per_school_summary` with columns for the calculations above.
per_school_summary = pd.DataFrame({'School Name':school_types['school_name'],
                                   'School Types':school_types['type'],
                                   'Students per school':per_school_counts['student_count'], 
                                   'Total School Budget':per_school_budget['school_budget'],
                                   'Per Student Budget':per_school_capita['Per Student Capita'],
                                   'Average Math score':per_school_math['math_score'],
                                   'Average Reading score':per_school_reading['reading_score'],
                                   'Percent Passing Math':per_school_passing_math['% Passing Math'],
                                   'Percent Passing Reading':per_school_passing_reading['% Passing Reading'],
                                   'Number Passing Both':school_students_passing_math_and_reading['# Passing Both'],
                                   'Percent Passing Both':overall_passing_rate['% Passing Both']})
 
#Formatting
#per_school_summary["Total School Budget"] = per_school_summary["Total School Budget"].map("${:,.2f}".format)
#per_school_summary["Per Student Budget"] = per_school_summary["Per Student Budget"].map("${:,.2f}".format)

# Display the DataFrame
per_school_summary


,School Name,School Types,Students per school,Total School Budget,Per Student Budget,Average Math score,Average Reading score,Percent Passing Math,Percent Passing Reading,Number Passing Both,Percent Passing Both
0,Bailey High School,District,4976,3124928,628.0,77.048432,81.033963,66.680064,81.933280,2719,54.642283
1,Cabrera High School,Charter,1858,1081356,582.0,83.061895,83.975780,94.133477,97.039828,1697,91.334769
2,Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,65.988471,80.739234,1569,53.204476
3,Ford High School,District,2739,1763916,644.0,77.102592,80.746258,68.309602,79.299014,1487,54.289887
4,Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,93.392371,97.138965,1330,90.599455
5,Hernandez High School,District,4635,3022020,652.0,77.289752,80.934412,66.752967,80.862999,2481,53.527508
6,Holden High School,Charter,427,248087,581.0,83.803279,83.814988,92.505855,96.252927,381,89.227166
7,Huang High School,District,2917,1910635,655.0,76.629414,81.182722,65.683922,81.316421,1561,53.513884
8,Johnson High School,District,4761,3094650,650.0,77.072464,80.966394,66.057551,81.222432,2549,53.539172
9,Pena High School,Charter,962,585858,609.0,83.839917,84.044699,94.594595,95.945946,871,90.540541


## Highest-Performing Schools (by % Overall Passing)

In [23]:
# Sort the schools by `% Overall Passing` in descending order and display the top 5 rows.
top_schools = per_school_summary.sort_values(['Percent Passing Both'], ascending=False)
top_schools.head(5)

,School Name,School Types,Students per school,Total School Budget,Per Student Budget,Average Math score,Average Reading score,Percent Passing Math,Percent Passing Reading,Number Passing Both,Percent Passing Both
1,Cabrera High School,Charter,1858,1081356,582.0,83.061895,83.975780,94.133477,97.039828,1697,91.334769
12,Thomas High School,Charter,1635,1043130,638.0,83.418349,83.848930,93.272171,97.308869,1487,90.948012
4,Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,93.392371,97.138965,1330,90.599455
13,Wilson High School,Charter,2283,1319574,578.0,83.274201,83.989488,93.867718,96.539641,2068,90.582567
9,Pena High School,Charter,962,585858,609.0,83.839917,84.044699,94.594595,95.945946,871,90.540541


## Bottom Performing Schools (By % Overall Passing)

In [24]:
# Sort the schools by `% Overall Passing` in ascending order and display the top 5 rows.
bottom_schools = per_school_summary.sort_values(['Percent Passing Both'], ascending=True)
bottom_schools.head(5)

,School Name,School Types,Students per school,Total School Budget,Per Student Budget,Average Math score,Average Reading score,Percent Passing Math,Percent Passing Reading,Number Passing Both,Percent Passing Both
10,Rodriguez High School,District,3999,2547363,637.0,76.842711,80.744686,66.366592,80.220055,2119,52.988247
2,Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,65.988471,80.739234,1569,53.204476
7,Huang High School,District,2917,1910635,655.0,76.629414,81.182722,65.683922,81.316421,1561,53.513884
5,Hernandez High School,District,4635,3022020,652.0,77.289752,80.934412,66.752967,80.862999,2481,53.527508
8,Johnson High School,District,4761,3094650,650.0,77.072464,80.966394,66.057551,81.222432,2549,53.539172


## Math Scores by Grade

In [25]:
# Use the code provided to separate the data by grade
ninth_graders = school_data_complete_df[(school_data_complete_df["grade"] == "9th")]
tenth_graders = school_data_complete_df[(school_data_complete_df["grade"] == "10th")]
eleventh_graders = school_data_complete_df[(school_data_complete_df["grade"] == "11th")]
twelfth_graders = school_data_complete_df[(school_data_complete_df["grade"] == "12th")]

# Group by `school_name` and take the mean of the `math_score` column for each.
ninth_grade_math_scores = ninth_graders.groupby('school_name')["math_score"].mean()
tenth_grader_math_scores = tenth_graders.groupby('school_name')["math_score"].mean()
eleventh_grader_math_scores = eleventh_graders.groupby('school_name')["math_score"].mean()
twelfth_grader_math_scores = twelfth_graders.groupby('school_name')["math_score"].mean()

# Combine each of the scores above into single DataFrame called `math_scores_by_grade`
# Make a dictionary, 
dict1 = {'9th Grade Math':ninth_grade_math_scores,
        '10th Grade Math':tenth_grader_math_scores,
        '11th Grade Math':eleventh_grader_math_scores,
        '12th Grade Math':twelfth_grader_math_scores}

# Make dict into DF
math_scores_by_grade = pd.DataFrame(dict1)

# Format scores
# Source: https://stackoverflow.com/questions/23981601/format-certain-floating-dataframe-columns-into-percentage-in-pandas
math_scores_by_grade = math_scores_by_grade.style.format({
    '9th Grade Math': '{:,.2f}%'.format,
    '10th Grade Math': '{:,.2f}%'.format,
    '11th Grade Math': '{:,.2f}%'.format,
    '12th Grade Math': '{:,.2f}%'.format})

# Minor data wrangling
math_scores_by_grade.index.name = None

# Display the DataFrame
math_scores_by_grade

,9th Grade Math,10th Grade Math,11th Grade Math,12th Grade Math
Bailey High School,77.08%,77.00%,77.52%,76.49%
Cabrera High School,83.09%,83.15%,82.77%,83.28%
Figueroa High School,76.40%,76.54%,76.88%,77.15%
Ford High School,77.36%,77.67%,76.92%,76.18%
Griffin High School,82.04%,84.23%,83.84%,83.36%
Hernandez High School,77.44%,77.34%,77.14%,77.19%
Holden High School,83.79%,83.43%,85.00%,82.86%
Huang High School,77.03%,75.91%,76.45%,77.23%
Johnson High School,77.19%,76.69%,77.49%,76.86%
Pena High School,83.63%,83.37%,84.33%,84.12%


## Reading Score by Grade 

In [26]:
# Use the code provided to separate the data by grade
ninth_graders = school_data_complete_df[(school_data_complete_df["grade"] == "9th")]
tenth_graders = school_data_complete_df[(school_data_complete_df["grade"] == "10th")]
eleventh_graders = school_data_complete_df[(school_data_complete_df["grade"] == "11th")]
twelfth_graders = school_data_complete_df[(school_data_complete_df["grade"] == "12th")]

# Group by `school_name` and take the mean of the the `reading_score` column for each.
ninth_grade_reading_scores = ninth_graders.groupby('school_name')["reading_score"].mean()
tenth_grader_reading_scores = tenth_graders.groupby('school_name')["reading_score"].mean()
eleventh_grader_reading_scores = eleventh_graders.groupby('school_name')["reading_score"].mean()
twelfth_grader_reading_scores = twelfth_graders.groupby('school_name')["reading_score"].mean()

# Combine each of the scores above into single DataFrame called `reading_scores_by_grade`
# Make a dictionary, 
dict2 = {'9th Grade Math':ninth_grade_reading_scores,
        '10th Grade Math':tenth_grader_reading_scores,
        '11th Grade Math':eleventh_grader_reading_scores,
        '12th Grade Math':twelfth_grader_reading_scores}

# Make dict into DF
reading_scores_by_grade = pd.DataFrame(dict2)

# Format scores
# Source: https://stackoverflow.com/questions/23981601/format-certain-floating-dataframe-columns-into-percentage-in-pandas
reading_scores_by_grade = reading_scores_by_grade.style.format({
    '9th Grade Math': '{:,.2f}%'.format,
    '10th Grade Math': '{:,.2f}%'.format,
    '11th Grade Math': '{:,.2f}%'.format,
    '12th Grade Math': '{:,.2f}%'.format})

# Minor data wrangling
#reading_scores_by_grade = reading_scores_by_grade[["9th", "10th", "11th", "12th"]]
#reading_scores_by_grade.index.name = None

# Display the DataFrame
reading_scores_by_grade

,9th Grade Math,10th Grade Math,11th Grade Math,12th Grade Math
school_name,,,,
Bailey High School,81.30%,80.91%,80.95%,80.91%
Cabrera High School,83.68%,84.25%,83.79%,84.29%
Figueroa High School,81.20%,81.41%,80.64%,81.38%
Ford High School,80.63%,81.26%,80.40%,80.66%
Griffin High School,83.37%,83.71%,84.29%,84.01%
Hernandez High School,80.87%,80.66%,81.40%,80.86%
Holden High School,83.68%,83.32%,83.82%,84.70%
Huang High School,81.29%,81.51%,81.42%,80.31%
Johnson High School,81.26%,80.77%,80.62%,81.23%


## Scores by School Spending

In [27]:
# Establish the bins 
spending_bins = [0, 585, 630, 645, 680]
bin_labels = ["<$585", "$585-630", "$630-645", "$645-680"]

In [28]:
# Create a copy of the school summary since it has the "Per Student Budget" 
school_spending_df = per_school_summary.copy()

In [29]:
# Use `pd.cut` to categorize spending based on the bins.
school_spending_df["Spending Ranges (Per Student)"] = pd.cut(per_school_summary['Per Student Budget'],
                                                             spending_bins, labels=bin_labels)
#del school_spending_df['School Types']
#del school_spending_df['Students per school']
#del school_spending_df['Total School Budget']
#del school_spending_df['Average Math score']
#del school_spending_df['Average Reading score']
#del school_spending_df['Percent Passing Math']
#del school_spending_df['Percent Passing Reading']
#del school_spending_df['Number Passing Both']
#del school_spending_df['Percent Passing Both']
school_spending_df

,School Name,School Types,Students per school,Total School Budget,Per Student Budget,Average Math score,Average Reading score,Percent Passing Math,Percent Passing Reading,Number Passing Both,Percent Passing Both,Spending Ranges (Per Student)
0,Bailey High School,District,4976,3124928,628.0,77.048432,81.033963,66.680064,81.933280,2719,54.642283,$585-630
1,Cabrera High School,Charter,1858,1081356,582.0,83.061895,83.975780,94.133477,97.039828,1697,91.334769,<$585
2,Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,65.988471,80.739234,1569,53.204476,$630-645
3,Ford High School,District,2739,1763916,644.0,77.102592,80.746258,68.309602,79.299014,1487,54.289887,$630-645
4,Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,93.392371,97.138965,1330,90.599455,$585-630
5,Hernandez High School,District,4635,3022020,652.0,77.289752,80.934412,66.752967,80.862999,2481,53.527508,$645-680
6,Holden High School,Charter,427,248087,581.0,83.803279,83.814988,92.505855,96.252927,381,89.227166,<$585
7,Huang High School,District,2917,1910635,655.0,76.629414,81.182722,65.683922,81.316421,1561,53.513884,$645-680
8,Johnson High School,District,4761,3094650,650.0,77.072464,80.966394,66.057551,81.222432,2549,53.539172,$645-680
9,Pena High School,Charter,962,585858,609.0,83.839917,84.044699,94.594595,95.945946,871,90.540541,$585-630


In [30]:
#  Calculate averages for the desired columns. 
spending_math_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"])["Average Math score"].mean()
spending_reading_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"])["Average Reading score"].mean()
spending_passing_math = school_spending_df.groupby(["Spending Ranges (Per Student)"])["Percent Passing Math"].mean()
spending_passing_reading = school_spending_df.groupby(["Spending Ranges (Per Student)"])["Percent Passing Reading"].mean()
overall_passing_spending = school_spending_df.groupby(["Spending Ranges (Per Student)"])["Percent Passing Both"].mean()
spending_passing_reading

C:\Users\Khrystyne\AppData\Local\Temp\ipykernel_2276\1462530326.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  spending_math_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"])["Average Math score"].mean()
C:\Users\Khrystyne\AppData\Local\Temp\ipykernel_2276\1462530326.py:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  spending_reading_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"])["Average Reading score"].mean()
C:\Users\Khrystyne\AppData\Local\Temp\ipykernel_2276\1462530326.py:4: FutureWarning: The default of observed=False is deprecated and will be changed t

Spending Ranges (Per Student)
<$585       96.610877
$585-630    92.718205
$630-645    84.391793
$645-680    81.133951
Name: Percent Passing Reading, dtype: float64

In [31]:
# Assemble into DataFrame
spending_summary = pd.DataFrame(school_spending_df)

# Display results
spending_summary

,School Name,School Types,Students per school,Total School Budget,Per Student Budget,Average Math score,Average Reading score,Percent Passing Math,Percent Passing Reading,Number Passing Both,Percent Passing Both,Spending Ranges (Per Student)
0,Bailey High School,District,4976,3124928,628.0,77.048432,81.033963,66.680064,81.933280,2719,54.642283,$585-630
1,Cabrera High School,Charter,1858,1081356,582.0,83.061895,83.975780,94.133477,97.039828,1697,91.334769,<$585
2,Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,65.988471,80.739234,1569,53.204476,$630-645
3,Ford High School,District,2739,1763916,644.0,77.102592,80.746258,68.309602,79.299014,1487,54.289887,$630-645
4,Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,93.392371,97.138965,1330,90.599455,$585-630
5,Hernandez High School,District,4635,3022020,652.0,77.289752,80.934412,66.752967,80.862999,2481,53.527508,$645-680
6,Holden High School,Charter,427,248087,581.0,83.803279,83.814988,92.505855,96.252927,381,89.227166,<$585
7,Huang High School,District,2917,1910635,655.0,76.629414,81.182722,65.683922,81.316421,1561,53.513884,$645-680
8,Johnson High School,District,4761,3094650,650.0,77.072464,80.966394,66.057551,81.222432,2549,53.539172,$645-680
9,Pena High School,Charter,962,585858,609.0,83.839917,84.044699,94.594595,95.945946,871,90.540541,$585-630


## Scores by School Size

In [32]:
# Establish the bins.
size_bins = [0, 1000, 2000, 5000]
labels = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

In [33]:
# Categorize the spending based on the bins
# Use `pd.cut` on the "Total Students" column of the `per_school_summary` DataFrame.
per_school_summary["School Size"] = pd.cut(per_school_summary['Students per school'],
                                                             size_bins, labels=labels)
per_school_summary

,School Name,School Types,Students per school,Total School Budget,Per Student Budget,Average Math score,Average Reading score,Percent Passing Math,Percent Passing Reading,Number Passing Both,Percent Passing Both,School Size
0,Bailey High School,District,4976,3124928,628.0,77.048432,81.033963,66.680064,81.933280,2719,54.642283,Large (2000-5000)
1,Cabrera High School,Charter,1858,1081356,582.0,83.061895,83.975780,94.133477,97.039828,1697,91.334769,Medium (1000-2000)
2,Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,65.988471,80.739234,1569,53.204476,Large (2000-5000)
3,Ford High School,District,2739,1763916,644.0,77.102592,80.746258,68.309602,79.299014,1487,54.289887,Large (2000-5000)
4,Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,93.392371,97.138965,1330,90.599455,Medium (1000-2000)
5,Hernandez High School,District,4635,3022020,652.0,77.289752,80.934412,66.752967,80.862999,2481,53.527508,Large (2000-5000)
6,Holden High School,Charter,427,248087,581.0,83.803279,83.814988,92.505855,96.252927,381,89.227166,Small (<1000)
7,Huang High School,District,2917,1910635,655.0,76.629414,81.182722,65.683922,81.316421,1561,53.513884,Large (2000-5000)
8,Johnson High School,District,4761,3094650,650.0,77.072464,80.966394,66.057551,81.222432,2549,53.539172,Large (2000-5000)
9,Pena High School,Charter,962,585858,609.0,83.839917,84.044699,94.594595,95.945946,871,90.540541,Small (<1000)


In [34]:
# Calculate averages for the desired columns. 
size_math_scores = per_school_summary.groupby(["School Size"])["Average Math score"].mean()
size_reading_scores = per_school_summary.groupby(["School Size"])["Average Reading score"].mean()
size_passing_math = per_school_summary.groupby(["School Size"])["Percent Passing Math"].mean()
size_passing_reading = per_school_summary.groupby(["School Size"])["Percent Passing Reading"].mean()
size_overall_passing = per_school_summary.groupby(["School Size"])["Percent Passing Both"].mean()
size_overall_passing

C:\Users\Khrystyne\AppData\Local\Temp\ipykernel_2276\4008611108.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  size_math_scores = per_school_summary.groupby(["School Size"])["Average Math score"].mean()
C:\Users\Khrystyne\AppData\Local\Temp\ipykernel_2276\4008611108.py:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  size_reading_scores = per_school_summary.groupby(["School Size"])["Average Reading score"].mean()
C:\Users\Khrystyne\AppData\Local\Temp\ipykernel_2276\4008611108.py:4: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass o

School Size
Small (<1000)         89.883853
Medium (1000-2000)    90.621535
Large (2000-5000)     58.286003
Name: Percent Passing Both, dtype: float64

In [35]:
# Create a DataFrame called `size_summary` that breaks down school performance based on school size (small, medium, or large).
# Use the scores above to create a new DataFrame called `size_summary`
size_summary = pd.DataFrame(per_school_summary.copy())

# Display results
size_summary

,School Name,School Types,Students per school,Total School Budget,Per Student Budget,Average Math score,Average Reading score,Percent Passing Math,Percent Passing Reading,Number Passing Both,Percent Passing Both,School Size
0,Bailey High School,District,4976,3124928,628.0,77.048432,81.033963,66.680064,81.933280,2719,54.642283,Large (2000-5000)
1,Cabrera High School,Charter,1858,1081356,582.0,83.061895,83.975780,94.133477,97.039828,1697,91.334769,Medium (1000-2000)
2,Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,65.988471,80.739234,1569,53.204476,Large (2000-5000)
3,Ford High School,District,2739,1763916,644.0,77.102592,80.746258,68.309602,79.299014,1487,54.289887,Large (2000-5000)
4,Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,93.392371,97.138965,1330,90.599455,Medium (1000-2000)
5,Hernandez High School,District,4635,3022020,652.0,77.289752,80.934412,66.752967,80.862999,2481,53.527508,Large (2000-5000)
6,Holden High School,Charter,427,248087,581.0,83.803279,83.814988,92.505855,96.252927,381,89.227166,Small (<1000)
7,Huang High School,District,2917,1910635,655.0,76.629414,81.182722,65.683922,81.316421,1561,53.513884,Large (2000-5000)
8,Johnson High School,District,4761,3094650,650.0,77.072464,80.966394,66.057551,81.222432,2549,53.539172,Large (2000-5000)
9,Pena High School,Charter,962,585858,609.0,83.839917,84.044699,94.594595,95.945946,871,90.540541,Small (<1000)


## Scores by School Type

In [36]:
# Group the per_school_summary DataFrame by "School Type" and average the results.
average_math_score_by_type = per_school_summary.groupby(["School Types"])["Average Math score"].mean()
average_reading_score_by_type = per_school_summary.groupby(["School Types"])["Average Reading score"].mean()
average_percent_passing_math_by_type = per_school_summary.groupby(["School Types"])["Percent Passing Math"].mean()
average_percent_passing_reading_by_type = per_school_summary.groupby(["School Types"])["Percent Passing Reading"].mean()
average_percent_overall_passing_by_type = per_school_summary.groupby(["School Types"])["Percent Passing Both"].mean()
average_percent_overall_passing_by_type

School Types
Charter     90.432244
District    53.672208
Name: Percent Passing Both, dtype: float64

In [37]:
# Assemble the new data by type into a DataFrame called `type_summary`
type_summary = pd.DataFrame({'Avg Math Score': average_math_score_by_type,
                            'Avg Reading Score': average_reading_score_by_type,
                            'Passing Math Percent': average_percent_passing_math_by_type,
                            'Passing Reading Percent': average_percent_passing_reading_by_type,
                            'Passing Both Percent': average_percent_overall_passing_by_type})

# Display results
type_summary

,Avg Math Score,Avg Reading Score,Passing Math Percent,Passing Reading Percent,Passing Both Percent
School Types,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,90.432244
District,76.956733,80.966636,66.548453,80.799062,53.672208
